In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-t9jb8np2
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-t9jb8np2
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Using cached transformers-4.8.1-py3-none-any.whl (2.5 MB)
  Using cached tokenizers-0.10.3-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (3.3 MB)
  Using cached huggingface_hub-0.0.12-py3-none-any.whl (37 kB)
  Using cached urllib3-1.25.11-py2.py3-none-any.whl (127 kB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.1
    Uninstalling tokenizers-0.13.1:
      Successfully uninstalled tokenizers-0.13.1
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.12
    Uninstalling urllib3-1.26.12:
      Succ

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [3]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [4]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

2023-03-02 11:35:40.774759: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [5]:
##GPU 사용 시
device = torch.device("cuda:0")

In [6]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /home/mglee/VSCODE/git_folder/KakaoPay_ESG/code/news/.cache/kobert_v1.zip
using cached model. /home/mglee/VSCODE/git_folder/KakaoPay_ESG/code/news/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [18]:
#기존 방식은 네이버 영화 리뷰로 학습한 것
# 15만건의 학습 데이터
# !wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
# !wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

In [7]:
import pandas as pd
KP_data = pd.read_excel("../../data/comments_label/KP_senti.xlsx")

In [8]:
temp = KP_data.sample(frac=1).reset_index(drop=True)
KP_train = temp[:1500]
KP_test = temp[1500:]

In [9]:
KP_train.to_csv('../../data/comments_label/KP_train.tsv', index=False, header=None, sep = "\t")
KP_test.to_csv('../../data/comments_label/KP_test.tsv', index=False, header=None, sep = "\t")

In [10]:
dataset_train = nlp.data.TSVDataset("../../data/comments_label/KP_train.tsv", field_indices=[0,1], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset("../../data/comments_label/KP_test.tsv", field_indices=[0,1], num_discard_samples=1)

In [11]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /home/mglee/VSCODE/git_folder/KakaoPay_ESG/code/news/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [12]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [35]:
## Setting parameters
max_len = 64
batch_size = 32
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 500
learning_rate =  5e-5

In [36]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [37]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [38]:
for data in train_dataloader:
  print("Data: ", data)
  print("Waveform: {}\nSample rate: {}\nLabels: {}".format(data[0], data[1], data[2]))
  break

Data:  [tensor([[   2, 4937, 6896,  ...,    1,    1,    1],
        [   2, 4640,  987,  ...,    1,    1,    1],
        [   2, 4640, 4241,  ...,    1,    1,    1],
        ...,
        [   2, 1370, 7295,  ...,    1,    1,    1],
        [   2,  518,  518,  ...,    1,    1,    1],
        [   2, 4640, 5330,  ...,    1,    1,    1]], dtype=torch.int32), tensor([48,  9, 12, 11, 64, 64, 21, 22, 45,  8, 15, 14, 64,  7, 26, 12, 36, 13,
        64, 33, 20, 14,  7,  6, 43, 41, 24, 49, 49, 28, 39,  9],
       dtype=torch.int32), tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32), tensor([0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1,
        0, 0, 0, 0, 1, 0, 0, 1], dtype=torch.int32)]
Waveform: tensor([[   2, 4937, 6896,  ...,    1,    1,    1],
        [   2, 4640,  987,  ...,    1,   

In [39]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [40]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [41]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [42]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [43]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [44]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [45]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [46]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/tmp/ipykernel_29318/1280546429.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/47 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.8947916030883789 train acc 0.15625
epoch 1 train acc 0.8251329787234043


/tmp/ipykernel_29318/1280546429.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/6 [00:00<?, ?it/s]

...

epoch 4 test acc 0.8315972222222222


  0%|          | 0/47 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.0024776419159024954 train acc 1.0
epoch 5 train acc 1.0


  0%|          | 0/6 [00:00<?, ?it/s]

epoch 5 test acc 0.8368055555555555


In [47]:
torch.save(model, '/home/mglee/VSCODE/git_folder/BERT/Sentiment/KP_KoBERT_Sentiment_e10_230302.pt')  # 전체 모델 저장
torch.save(model.state_dict(), '/home/mglee/VSCODE/git_folder/BERT/Sentiment/KP_Kobert_Sentiment_e10_state_dict_230302.pt')  # 모델 객체의 state_dict 저장
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, '/home/mglee/VSCODE/git_folder/BERT/Sentiment/KP_Kobert_Sentiment_all.tar') 

In [ ]:
PATH = '../result/kobert/'

pred_model = torch.load(PATH + 'KoBERT_0705_e15_py.pt')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수
# pred_model.load_state_dict(torch.load(PATH + 'Kobert_0627_py_state_dict.pt'))  # state_dict를 불러 온 후, 모델에 저장

# checkpoint = torch.load(PATH + 'all.tar')   # dict 불러오기
# pred_model.load_state_dict(checkpoint['model'])
#optimizer.load_state_dict(checkpoint['optimizer'])

### TEST DATA

In [61]:
import pandas as pd

posco = pd.read_csv("../../naver_comments/result/posco_token.csv")

In [104]:
test = posco.loc[:,['comment']]

In [105]:
test['label'] = 0

In [106]:
test

,comment,label
0,"""즈그말은 그렇지만 현장분위기는 완전 틀리던디""",0
1,"""포스코 화이팅 !! 가즈아""",0
2,"""또 거짓말 . 언플할 시간에 복구에 집중해라""",0
3,"""이 기회에 중간유통 마진이랑 업체들 정리 하시고\n스틸트레이드 같은 형식의 플랫폼...",0
4,"""역대 최악 최정우 되고 나서 재해사고\n엄청 났지..그냥 너는 포스코랑 안 맞다\...",0
...,...,...
13506,"""아~~자정까지 기다렸네...태풍구경할라구~~~""",0
13507,"""방송만 난리부르스고 수도권은 느낌도 안나네""",0
13508,"""삼성은 전자만""",0
13509,"""그냥 일해라 태풍이 뭐시라고 겁먹노 귀가찬다.""",0


In [107]:
class BERTDataset_test(Dataset):
    def __init__(self, dataset,bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([" ".join(str(dataset.iloc[i]['comment']))]) for i in range(len(dataset))]
        self.labels = [np.int32(dataset.iloc[i]['label']) for i in range(len(dataset))]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))



In [108]:
comment_test = BERTDataset_test(test, tok, max_len, True, False)

In [109]:
comment_test

In [110]:
pred_dataloader = torch.utils.data.DataLoader(comment_test, batch_size = batch_size, num_workers = 8)

In [114]:
for data in pred_dataloader:
  print("Data: ", data)
  print("Waveform: {}\nSample rate: {}\nLabels: {}".format(data[0], data[1], data[2]))
  break

Data:  [tensor([[   2,  518,  517,  ...,    1,    1,    1],
        [   2,  518, 4856,  ...,    1,    1,    1],
        [   2,  518, 1861,  ...,    1,    1,    1],
        ...,
        [   2,  518, 3552,  ...,    1,    1,    1],
        [   2,  518, 4457,  ...,    0,  517,    3],
        [   2,  518, 2718,  ..., 3758, 1562,    3]], dtype=torch.int32), tensor([29, 20, 26, 64, 60, 18, 17, 64, 23, 45, 64, 35, 64, 28, 35, 40, 17, 64,
        64, 35, 42, 28, 64, 59, 64, 35, 24, 20, 64, 17, 17, 34, 64, 64, 64, 29,
        64, 64, 61, 64, 52, 29, 64, 64, 13, 64, 64, 21, 30, 23, 64, 34,  7,  8,
        17, 47, 64, 64, 61, 64, 52, 29, 64, 64], dtype=torch.int32), tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32), tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0,

In [115]:
out_lst = []

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(pred_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)
    out_lst.append(out.data.cpu())
    max_vals, max_indices = torch.max(out, 1)

100%|██████████| 212/212 [00:08<00:00, 24.51it/s]


In [132]:
pred = []
for batch in out_lst:
    for item in batch:
        pred.append(np.argmax(item.numpy()))

In [135]:
test['label'] = pred

In [145]:
posco['sentiment'] = pred

In [149]:
posco.groupby('sentiment').size()

sentiment
0    11930
1     1581
dtype: int64

In [148]:
posco.to_csv("../../naver_comments/result/naver_POSCO_sentiment.csv")